## Integrating MCP Tool Calls Using MCP Server

In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

# Load your OPENAI API key from the .env file
load_dotenv()

# Initialize the chat model with the specified model name
llm = init_chat_model("openai:gpt-4o")

In [2]:
from typing import Annotated
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_mcp_adapters.client import MultiServerMCPClient

# Creates our LLM with calculator tools
async def create_mcp_graph():
    # Intialize the MCP client to connect to the calculator server
    # MCP (Model Context Protocol) allows LLMs to interact with external tools

    client = MultiServerMCPClient({
        "math": {
            "command": "python", # running a python script
            "args": ["calculator_server.py"], # filepath to the calculator server script
            "transport": "stdio" # communicate via stdin and stdout
        }
    })
    
    # Discover tools from MCP server
    mcp_tools = await client.get_tools()
    print(f"Found MCP tools: {[tool.name for tool in mcp_tools]}")
    
    # Bind the found tools to the LLM
    # Telling the LLM to use these tools when needed
    llm_with_tools = llm.bind_tools(mcp_tools)
    
    # Defining the state structure for our graph
    # Messages accumulate as the conversation progresses
    class State(TypedDict):
        messages: Annotated[list, add_messages]
    
    # Define the chatbot that processes messages
    # Must be async to work with MCP server because of tool calls over network
    async def chatbot(state: State):  # ← Must be async for MCP
        return {"messages": [await llm_with_tools.ainvoke(state["messages"])]}
    
    # Create the state graph
    graph_builder = StateGraph(State)

    # Add the chatbot node that decides what to do
    graph_builder.add_node("chatbot", chatbot)
    
    # Add the tool node that executes the calculator tools
    tool_node = ToolNode(tools=mcp_tools)
    graph_builder.add_node("tools", tool_node)
    
    # Conditional routing from chatbot to tools
    # If the chatbot decides a tool is needed, it will route to the tools node
    # If the tool is not needed, it will end the conversation
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
        {"tools": "tools", END: END}
    )
    
    # After the tools execute, return to the chatbot to process the result
    graph_builder.add_edge("tools", "chatbot")

    # Start the graph with the initial chatbot node
    graph_builder.add_edge(START, "chatbot")
    
    return graph_builder.compile()


# Cleanup function to close MCP connections
async def cleanup():
    """Simple cleanup when done with demo"""
    try:
        if 'graph' in globals():
            # Close any MCP connections if they exist
            print("🧹 Demo cleanup complete")
    except Exception as e:
        print(f"Cleanup error: {e}")

# Call when you're done:
# await cleanup()


# Create the calculator graph by connecting to MCP server
graph = await create_mcp_graph()



Found MCP tools: ['square_number', 'add_numbers', 'subtract_numbers', 'multiply_numbers', 'divide_numbers', 'power', 'square_root']


In [3]:
async def calculate_with_validation(query: str):
    """Calculate with input validation and better error handling"""

    # Check if the query is meaningful
    if not query or len(query.strip()) < 3:
        return "Please provide a valid math question."
    
    try:
        # Invoke the graph with a system message to guid the LLM and user message with the query
        result = await graph.ainvoke({
            "messages": [
                {
                    # system message instructs the LLM on how to behave
                    "role": "system",
                    "content": "You are a precise calculator. Follow order of operations strictly. Use parentheses to clarify operations. Always calculate step by step."
                },
                {
                    # user message contains the math query
                    "role": "user",
                    "content": query
                }
            ]
        })
        # Return the result from the LLM
        return result
    except Exception as e:
        return f"Calculation error: {str(e)}"


In [4]:
# Example usage
result = await calculate_with_validation("what's the square root of 156.789, plus 47.234, then multiply by 3?")

In [5]:
# Viewing the answer
# Output is an AIMessage
# Accessing the last message content for the final answer
print("Result:", result["messages"][-1].content)

Result: The result of the calculation \(((\sqrt{156.789} + 47.234) \times 3)\) is approximately \(179.27\).


In [7]:
# Function to visualize the tool flow
def quick_tool_flow(result):
    """Quick tool flow view with each call on a new line."""
    
    # getting all messages from the conversation result
    messages = result["messages"]
    
    flow = []

    # go through each message and check for tool calls
    for message in messages:
        # checking tool call
        if hasattr(message, 'tool_calls') and message.tool_calls:
            for call in message.tool_calls:
                flow.append(f"🔧 {call['name']}({call['args']})")
        # checking tool response
        elif hasattr(message, 'name') and hasattr(message, 'content'):
            flow.append(f"→ {message.content}")
    
    print("TOOL CALL FLOW:")
    for step in flow:
        print(step)

In [8]:
# Visualize the tool flow
quick_tool_flow(result)

TOOL CALL FLOW:
→ You are a precise calculator. Follow order of operations strictly. Use parentheses to clarify operations. Always calculate step by step.
→ what's the square root of 156.789, plus 47.234, then multiply by 3?
🔧 square_root({'number': 156.789})
→ 12.521541438656824
🔧 add_numbers({'a': 12.521541438656824, 'b': 47.234})
→ 59.75554143865683
🔧 multiply_numbers({'a': 59.75554143865683, 'b': 3})
→ 179.2666243159705
→ The result of the calculation \(((\sqrt{156.789} + 47.234) \times 3)\) is approximately \(179.27\).


In [9]:
await cleanup()  # Call cleanup when done

🧹 Demo cleanup complete
